In [1]:
import pandas as pd
ratings_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-100k/u.data', sep = '\t', names = ratings_cols, usecols = range(3))

movie_cols = ['movie_id', 'title']
movies = pd.read_csv('ml-100k/u.item', sep = '|', names = movie_cols, usecols = range(2), encoding = "ISO-8859-1")

ratings = pd.merge(movies, ratings)

ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [38]:
movie_ratings = ratings.pivot_table(index = ['title'], columns = ['user_id'], values = 'rating')
movie1_ratings = movie_ratings
movie_ratings.head(10)

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),5.0,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187 (1997),NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2 Days in the Valley (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
"20,000 Leagues Under the Sea (1954)",3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001: A Space Odyssey (1968),4.0,NaN,NaN,NaN,4.0,5.0,5.0,NaN,NaN,5.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
3 Ninjas: High Noon At Mega Mountain (1998),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#Nomalizing the Rows.
for i in range(movie_ratings.shape[0]):
    a = movie_ratings.iloc[i,:].sum(axis=0)
    b = movie_ratings.iloc[i,:].notnull().sum()
    #print("userId:{} avgRating: {}".format(i+1,a/b))
    k = a/b
    for j in range(movie_ratings.shape[1]):
        if movie_ratings.iloc[i,j] > 0:
            movie_ratings.iloc[i,j] = movie_ratings.iloc[i,j] - k
                   

In [32]:
movie_ratings.iloc[0,:].sum(axis=0)

-1.3322676295501878e-15

In [29]:
#Let's predict the rating fot the first movie i.e 'Til There Was You (1997)' for the 1st User.
#Therefore, we need to find the similarity between the movies
import numpy as np

def takesec(num):
    return num[1]

cosine_vector = []
num = 0
l1 = 0
l2 = 0
for i in range(1,movie_ratings.shape[0]):
    for j in range(movie_ratings.shape[1]):
        if movie_ratings.iloc[i,j] > 0 and movie_ratings.iloc[0,j] > 0:
            num = num + (movie_ratings.iloc[i,j] * movie_ratings.iloc[0,j])
        if movie_ratings.iloc[i,j] > 0:
            l1 = l1 + (movie_ratings.iloc[i,j] * movie_ratings.iloc[i,j])
        if movie_ratings.iloc[0,j] > 0:
            l2 = l2 + (movie_ratings.iloc[0,j] * movie_ratings.iloc[0,j])
    eventual_prod = np.sqrt(l1) * np.sqrt(l2)
    eventual_div = num/eventual_prod
    cosine_vector.append([i,eventual_div])
    
cosine_vector.sort(key=takesec, reverse=True)    
   
    

In [70]:
cosine_vector[:30]

[[26, 0.059687757563964085],
 [21, 0.05941897904114937],
 [18, 0.05898239012930128],
 [22, 0.057520662950151076],
 [19, 0.05731348809281697],
 [27, 0.05698715808574092],
 [28, 0.055641504472628264],
 [23, 0.055068805873707063],
 [25, 0.0540483075516364],
 [20, 0.053942119332877354],
 [29, 0.05358702833872409],
 [30, 0.05268634186677522],
 [24, 0.05175253181964582],
 [32, 0.050623014993208405],
 [5, 0.05053367434193741],
 [31, 0.05049228212692329],
 [33, 0.04980376754188194],
 [36, 0.0489353400055089],
 [17, 0.048827098796373695],
 [34, 0.04866243986398749],
 [37, 0.04799124085853266],
 [35, 0.0473595833430449],
 [38, 0.04716167407712662],
 [55, 0.046736982459723984],
 [39, 0.04653249122698487],
 [56, 0.04630413557406377],
 [59, 0.04604223791441976],
 [40, 0.0459502442864056],
 [60, 0.04565694044682837],
 [57, 0.045386806391734695]]

In [72]:
#Let's consider the top 50 similar rated movies and predict the output.

predict_movie = int(input ("Enter a UserId within range  0 to 942 : "))
count = 0
num1 = 0
den1 = 0
for i in cosine_vector:
    if movie1_ratings.iloc[i[0],predict_movie] > 0:
            count = count + 1
            num1 = num1 + i[1] * movie1_ratings.iloc[i[0],predict_movie]
            den1 = den1 + i[1]
        
    if count == 50:
        print ("Reached 50 :)")
        break

print("Expected Rating for 1st Movie for UserId: {} is :{}".format(predict_movie,num1/den1))        

Enter a UserId within range  0 to 942 : 151
Reached 50 :)
Expected Rating for 1st Movie for UserId: 151 is :4.280617826639626


In [47]:
for i in range(movie1_ratings.shape[1]):
    if movie1_ratings.iloc[0,i] > 0:
        print("Id = {} , Rating = {}".format(i,movie1_ratings.iloc[0,i]))

Id = 151 , Rating = 4.0
Id = 177 , Rating = 3.0
Id = 222 , Rating = 1.0
Id = 298 , Rating = 2.0
Id = 341 , Rating = 1.0
Id = 415 , Rating = 3.0
Id = 529 , Rating = 2.0
Id = 531 , Rating = 3.0
Id = 781 , Rating = 2.0


In [ ]:
We can observe that for user 151 the rating observed = 4.0 and the Predicted rating = 4.3 which is fairly close.

Therefore we can conclude that our method is Fairly Accurate.